<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/cosy_voice_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!nvidia-smi

Wed Jul 10 11:36:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0              32W /  70W |   2153MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.30
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [1]:
!git clone --recursive https://github.com/FunAudioLLM/CosyVoice.git


Cloning into 'CosyVoice'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 316 (delta 58), reused 48 (delta 43), pack-reused 229
Receiving objects: 100% (316/316), 767.13 KiB | 29.50 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Submodule 'third_party/Matcha-TTS' (https://github.com/shivammehta25/Matcha-TTS.git) registered for path 'third_party/Matcha-TTS'
Cloning into '/content/CosyVoice/third_party/Matcha-TTS'...
remote: Enumerating objects: 916, done.        
remote: Counting objects: 100% (306/306), done.        
remote: Compressing objects: 100% (92/92), done.        
remote: Total 916 (delta 257), reused 215 (delta 214), pack-reused 610        
Receiving objects: 100% (916/916), 64.08 MiB | 11.15 MiB/s, done.
Resolving deltas: 100% (416/416), done.
Submodule path 'third_party/Matcha-TTS': checked out 'dd9105b34bf2be2230f4aa1e4769fb586a3c824e'


In [2]:
!cd CosyVoice && pip install -r requirements.txt -i https://mirrors.aliyun.com/pypi/simple/ --trusted-host=mirrors.aliyun.com


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/, https://download.pytorch.org/whl/cu118
Ignoring onnxruntime: markers 'sys_platform == "darwin" or sys_platform == "windows"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.0/260.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!git lfs install

Git LFS initialized.


In [4]:
%cd /content/CosyVoice
# git模型下载，请确保已安装git lfs
#!rm -rf pretrained_models
!mkdir -p pretrained_models
!git clone https://www.modelscope.cn/iic/CosyVoice-300M.git pretrained_models/CosyVoice-300M
!git clone https://www.modelscope.cn/iic/CosyVoice-300M-SFT.git pretrained_models/CosyVoice-300M-SFT
!git clone https://www.modelscope.cn/iic/CosyVoice-300M-Instruct.git pretrained_models/CosyVoice-300M-Instruct
!git clone https://www.modelscope.cn/iic/CosyVoice-ttsfrd.git pretrained_models/CosyVoice-ttsfrd

/content/CosyVoice
Cloning into 'pretrained_models/CosyVoice-300M'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (20/20), 7.66 KiB | 7.66 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Filtering content: 100% (6/6), 2.13 GiB | 62.22 MiB/s, done.
Cloning into 'pretrained_models/CosyVoice-300M-SFT'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 20 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (20/20), 7.72 KiB | 7.72 MiB/s, done.
Resolving deltas: 100% (2/2), done.
Filtering content: 100% (6/6), 2.13 GiB | 48.68 MiB/s, done.
Cloning into 'pretrained_models/CosyVoice-300M-Instruct'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing 

In [5]:
!cd pretrained_models/CosyVoice-ttsfrd/ && wget https://www.modelscope.cn/models/speech_tts/speech_kantts_ttsfrd/resolve/master/ttsfrd-0.3.9-cp310-cp310-linux_x86_64.whl

--2024-07-10 11:21:08--  https://www.modelscope.cn/models/speech_tts/speech_kantts_ttsfrd/resolve/master/ttsfrd-0.3.9-cp310-cp310-linux_x86_64.whl
Resolving www.modelscope.cn (www.modelscope.cn)... 39.99.245.74, 39.98.76.40, 39.99.230.9, ...
Connecting to www.modelscope.cn (www.modelscope.cn)|39.99.245.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4005795 (3.8M) [application/octet-stream]
Saving to: ‘ttsfrd-0.3.9-cp310-cp310-linux_x86_64.whl’

ttsfrd-0.3.9-cp310- 100%[===================>]   3.82M  4.90MB/s    in 0.8s    

2024-07-10 11:21:09 (4.90 MB/s) - ‘ttsfrd-0.3.9-cp310-cp310-linux_x86_64.whl’ saved [4005795/4005795]



In [6]:
!ls pretrained_models/CosyVoice-ttsfrd/

configuration.json  ttsfrd-0.3.6-cp38-cp38-linux_x86_64.whl
README.md	    ttsfrd-0.3.9-cp310-cp310-linux_x86_64.whl
resource.zip


In [7]:
!cd pretrained_models/CosyVoice-ttsfrd/ && unzip resource.zip -d .
!pip install pretrained_models/CosyVoice-ttsfrd/ttsfrd-0.3.9-cp310-cp310-linux_x86_64.whl


Archive:  resource.zip
   creating: ./resource/
   creating: ./resource/ws_zhsc/
  inflating: ./resource/ws_zhsc/xiaoyan.dict  
  inflating: ./resource/ws_zhsc/ws-dfa-rule.dict  
  inflating: ./resource/ws_zhsc/ivr.dict  
  inflating: ./resource/ws_zhsc/crf_model  
  inflating: ./resource/ws_zhsc/pos_hmm.dict  
  inflating: ./resource/ws_zhsc/AliTokenizer_example.conf  
  inflating: ./resource/ws_zhsc/puncNumberTimeLetter.dict  
   creating: ./resource/jprsc/
  inflating: ./resource/jprsc/pos-id.def  
  inflating: ./resource/jprsc/rewrite.def  
  inflating: ./resource/jprsc/char.bin  
  inflating: ./resource/jprsc/right-id.def  
  inflating: ./resource/jprsc/matrix.bin  
  inflating: ./resource/jprsc/left-id.def  
  inflating: ./resource/jprsc/COPYING  
  inflating: ./resource/jprsc/sys.dic  
  inflating: ./resource/jprsc/unk.dic  
   creating: ./resource/ws_chhk/
  inflating: ./resource/ws_chhk/xiaoyan.dict  
  inflating: ./resource/ws_chhk/ws-dfa-rule.dict  
  inflating: ./resource/w

In [ ]:
!ls /content/CosyVoice/third_party/Matcha-TTS

configs  LICENSE   MANIFEST.in	notebooks	README.md	  scripts   synthesis.ipynb
data	 Makefile  matcha	pyproject.toml	requirements.txt  setup.py


In [8]:
import sys
print(sys.path)
sys.path.insert(1, "/content/CosyVoice/third_party/Matcha-TTS")


['/content', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [9]:
print(sys.path)


['/content', '/content/CosyVoice/third_party/Matcha-TTS', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [15]:
from cosyvoice.cli.cosyvoice import CosyVoice
from cosyvoice.utils.file_utils import load_wav
import torchaudio
import time

cosyvoice = CosyVoice('pretrained_models/CosyVoice-300M-SFT')
# sft usage
print(cosyvoice.list_avaliable_spks())
s = time.time()
output = cosyvoice.inference_sft('你好，我是通义生成式语音大模型，请问有什么可以帮您的吗？', '中文女')
print("sft",time.time()-s)
torchaudio.save('sft.wav', output['tts_speech'], 22050)

cosyvoice = CosyVoice('pretrained_models/CosyVoice-300M')
# zero_shot usage, <|zh|><|en|><|jp|><|yue|><|ko|> for Chinese/English/Japanese/Cantonese/Korean
prompt_speech_16k = load_wav('zero_shot_prompt.wav', 16000)
s = time.time()
output = cosyvoice.inference_zero_shot('收到好友从远方寄来的生日礼物，那份意外的惊喜与深深的祝福让我心中充满了甜蜜的快乐，笑容如花儿般绽放。', '希望你以后能够做的比我还好呦。', prompt_speech_16k)
print("zero_shot",time.time()-s)
torchaudio.save('zero_shot.wav', output['tts_speech'], 22050)
# cross_lingual usage
prompt_speech_16k = load_wav('cross_lingual_prompt.wav', 16000)
s = time.time()
output = cosyvoice.inference_cross_lingual('<|en|>And then later on, fully acquiring that company. So keeping management in line, interest in line with the asset that\'s coming into the family is a reason why sometimes we don\'t buy the whole thing.', prompt_speech_16k)
print("cross_lingual",time.time()-s)
torchaudio.save('cross_lingual.wav', output['tts_speech'], 22050)

cosyvoice = CosyVoice('pretrained_models/CosyVoice-300M-Instruct')
# instruct usage, support <laughter></laughter><strong></strong>[laughter][breath]
s = time.time()
output = cosyvoice.inference_instruct('在面对挑战时，他展现了非凡的<strong>勇气</strong>与<strong>智慧</strong>。', '中文男', 'Theo \'Crimson\', is a fiery, passionate rebel leader. Fights with fervor for justice, but struggles with impulsiveness.')
print("instruct",time.time()-s)
torchaudio.save('instruct.wav', output['tts_speech'], 22050)

2024-07-10 11:26:50,985 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:26:50,985 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:26:50,985 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:26:50,985 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
INFO:wetext-zh_normalizer:found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:26:50,990 WETEXT INFO                     /usr/local/lib/python3.10/dist-packages/tn/zh_tn_verbalizer.fst
2024-07-10 11:26:50,990 WETEXT INFO                     /usr/local/lib/python3.10/dist-packages/tn/zh_tn_verbalizer.fst
2024-07-10 11:26:50,990 WETEXT INFO                     /usr/local/lib/python3.10/dist-packages/tn/zh_tn_verbalizer.fst
2024-07-10 11:26:50,990 WETEXT INFO                     /usr/local/lib

['中文女', '中文男', '日语男', '粤语女', '英文女', '英文男', '韩语女']
sft 5.2784583568573


2024-07-10 11:27:06,818 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:27:06,818 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:27:06,818 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:27:06,818 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:27:06,818 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
INFO:wetext-zh_normalizer:found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:27:06,826 WETEXT INFO                     /usr/local/lib/python3.10/dist-packages/tn/zh_tn_verbalizer.fst
2024-07-10 11:27:06,826 WETEXT INFO                     /usr/local/lib/python3.10/dist-packages/tn/zh_tn_verbalizer.fst
2024-07-10 11:27:06,826 WETEXT INFO                     /usr/local/lib/pyt

zero_shot 15.467547178268433
min value is  tensor(-1.0054)
cross_lingual 27.18858551979065


2024-07-10 11:28:00,890 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:28:00,890 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:28:00,890 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:28:00,890 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:28:00,890 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:28:00,890 WETEXT INFO found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
INFO:wetext-zh_normalizer:found existing fst: /usr/local/lib/python3.10/dist-packages/tn/zh_tn_tagger.fst
2024-07-10 11:28:00,897 WETEXT INFO                     /usr/local/lib/python3.10/dist-packages/tn/zh_tn_verbalizer.fst
2024-07-10 11:28:00,897 WETEXT INFO                     /usr/local/lib/python3

instruct 5.891448974609375


In [11]:
from IPython.display import Audio
Audio("sft.wav", autoplay=True)


In [12]:
Audio("zero_shot.wav", autoplay=True)

In [13]:
Audio("cross_lingual.wav", autoplay=True)

In [14]:
Audio("instruct.wav", autoplay=True)

## 总结
- 通过torch在gpu上推理性能有待提高，离线场景可以支持，但是在线实时场景需要对tensor更好的+*操作加速
- 不支持流式输出结果